<a href="https://colab.research.google.com/github/dcontrerascaballol/inventos/blob/main/Copia_de_cartas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Instalación de paquetes para trabajar

 Se instalan paquetes para poder procesar los datos y el uso de archivos ubicados en las carpetas de trabajo.

In [ ]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 5.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import tempfile
from docx import Document

In [ ]:
# Ruta del documento Word genérico
ruta_word = "/content/drive/MyDrive/extracto2/plantilla_extracto_do_2.docx"

In [ ]:
datos = pd.read_excel("/content/drive/MyDrive/extracto2/extracto_2.xlsx")
print(datos.columns)

Index(['nombre_sostenedor', 'rut_sostenedor', 'n_rex', 'nombre_region', 'sep',
       'general', 'internado', 'mantenimiento', 'pie', 'proretencion',
       'refuerzo', 'total_general'],
      dtype='object')


In [ ]:
import os
# Directorio donde se guardarán las cartas
directorio_cartas = "/content/drive/MyDrive/extracto2"
os.makedirs(directorio_cartas, exist_ok=True)

### Se genera script para generar los extractos por sostenedor

El script busca incoporar a la planilla de word los datos de cada sostenedor que se encuentran en el excel, así mismo, se reproduce la n-veces necesarias, este proceso genera un documento word por cada sostenedor.

Se busca insertar el número de resolución, nombre de sostenedor, rut del sostenedor, nombre de la región, así como, los saldos finales por subvención y el total.

In [ ]:

# Función para reemplazar el texto en un documento Word
def reemplazar_texto(doc, marcador, reemplazo):
    for paragraph in doc.paragraphs:
        if marcador in paragraph.text:
            paragraph.text = paragraph.text.replace(marcador, reemplazo)
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                if marcador in cell.text:
                    cell.text = cell.text.replace(marcador, reemplazo)

# Función para crear un documento personalizado
def crear_carta(fila, ruta_word_generico, directorio_cartas):
    doc = Document(ruta_word_generico)

    # Reemplazar los marcadores en el documento
    reemplazar_texto(doc, "{{n_rex}}", str(fila['n_rex']))
    reemplazar_texto(doc, "{{nombre_sostenedor}}", str(fila['nombre_sostenedor']))
    reemplazar_texto(doc, "{{rut_sostenedor}}", str(fila['rut_sostenedor']))
    reemplazar_texto(doc, "{{nombre_region}}", str(fila['nombre_region']))
    reemplazar_texto(doc, "{{sep}}", str(fila['sep']))
    reemplazar_texto(doc, "{{general}}", str(fila['general']))
    reemplazar_texto(doc, "{{internado}}", str(fila['internado']))
    reemplazar_texto(doc, "{{mantenimiento}}", str(fila['mantenimiento']))
    reemplazar_texto(doc, "{{pie}}", str(fila['pie']))
    reemplazar_texto(doc, "{{proretencion}}", str(fila['proretencion']))
    reemplazar_texto(doc, "{{refuerzo}}", str(fila['refuerzo']))
    reemplazar_texto(doc, "{{total_general}}", str(fila['total_general']))


    # Guardar el documento
    file_path = os.path.join(directorio_cartas, f"{fila['nombre_sostenedor']}.docx")
    doc.save(file_path)
    return file_path


# Crear las cartas
archivos_cartas = []
for index, fila in datos.iterrows():
    carta = crear_carta(fila, ruta_word, directorio_cartas)
    archivos_cartas.append(carta)

# Guardar las rutas de las cartas creadas para su posterior uso
with open('archivos_cartas.txt', 'w') as f:
    for item in archivos_cartas:
        f.write("%s\n" % item)

Nos da como resultado, el n-veces de registros en la base de datos de sostenedores, en este caso, los que deben informarse mediante el diario oficial.


## Se requiere generar un documento resumen de los sostenedores que requieren se realice una restitución de fondos

Para hacer un resumen de los sostenedores, relevando, nombre, rut, región y resolución, de manera de enlistar los extractos que deben ser publicados en el diario oficial.

In [ ]:
pip install python-docx pandas openpyxl

In [ ]:
datos = pd.read_excel("/content/drive/MyDrive/listado/extracto.xlsx")
print(datos.columns)

Index(['N°', 'rut', 'Último periodo', 'Saldo total determinado', 'MultiRBD',
       'Lote', 'Medio de notificación', 'Fecha notificación ', 'Cómo Prodecer',
       'Dirección', 'Comuna', 'nombre_region', 'nombre_sostenedor',
       'rut_sostenedor', 'n_rex', 'Fechas_rex', 'sep', 'general', 'internado',
       'mantenimiento', 'pie', 'proretencion', 'refuerzo', 'total_general'],
      dtype='object')


In [ ]:
import pandas as pd
from docx import Document

# Leer los datos desde el archivo Excel
ruta_excel = "/content/drive/MyDrive/listado/extracto_2.xlsx"
datos = pd.read_excel(ruta_excel)

# Ruta de la plantilla Word
ruta_plantilla_word = "/content/drive/MyDrive/listado/LISTADO.docx"

# Documento unificado de salida
archivo_salida = "/content/drive/MyDrive/listado/Documento_Unificado_final.docx"

# Crear un documento Word vacío donde se agregarán las secciones repetidas
doc_unificado = Document()

# Función para reemplazar marcadores en los párrafos del documento
def reemplazar_marcadores_en_parrafos(doc, fila):
    for paragraph in doc.paragraphs:
        if paragraph.runs:  # Verificar si el párrafo tiene "runs"
            full_text = ''.join([run.text for run in paragraph.runs])
            new_text = full_text.replace("{{n_rex}}", str(fila['n_rex']))
            new_text = new_text.replace("{{nombre_sostenedor}}", str(fila['nombre_sostenedor']))
            new_text = new_text.replace("{{rut_sostenedor}}", str(fila['rut_sostenedor']))
            new_text = new_text.replace("{{nombre_region}}", str(fila['nombre_region']))

            # Limpiar el contenido anterior y reasignar el nuevo texto
            for run in paragraph.runs:
                run.text = ""
            paragraph.runs[0].text = new_text  # Asignar el nuevo texto al primer run

# Función para reemplazar marcadores en las tablas del documento (si existen)
def reemplazar_marcadores_en_tablas(doc, fila):
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for paragraph in cell.paragraphs:
                    if paragraph.runs:  # Verificar si el párrafo tiene "runs"
                        full_text = ''.join([run.text for run in paragraph.runs])
                        new_text = full_text.replace("{{n_rex}}", str(fila['n_rex']))
                        new_text = new_text.replace("{{nombre_sostenedor}}", str(fila['nombre_sostenedor']))
                        new_text = new_text.replace("{{rut_sostenedor}}", str(fila['rut_sostenedor']))
                        new_text = new_text.replace("{{nombre_region}}", str(fila['nombre_region']))

                        # Limpiar el contenido anterior y reasignar el nuevo texto
                        for run in paragraph.runs:
                            run.text = ""
                        paragraph.runs[0].text = new_text  # Asignar el nuevo texto al primer run

# Iterar sobre cada fila del archivo Excel
for index, fila in datos.iterrows():
    # Cargar la plantilla de Word
    doc_temp = Document(ruta_plantilla_word)

    # Reemplazar los marcadores con los datos de la fila actual
    reemplazar_marcadores_en_parrafos(doc_temp, fila)
    reemplazar_marcadores_en_tablas(doc_temp, fila)

    # Agregar el contenido del documento temporal al documento unificado
    for element in doc_temp.element.body:
        doc_unificado.element.body.append(element)

    # Añadir un salto de página después de cada sección
    doc_unificado.add_page_break()

# Eliminar el último salto de página si no es necesario
if doc_unificado.paragraphs[-1].text == '':
    doc_unificado.paragraphs[-1]._element.getparent().remove(doc_unificado.paragraphs[-1]._element)

# Guardar el documento unificado
doc_unificado.save(archivo_salida)
print(f"Documento unificado guardado en: {archivo_salida}")


Documento unificado guardado en: /content/drive/MyDrive/listado/Documento_Unificado_final.docx


El producto es un documento word que contiene n-veces la plantilla de sostenedores, por temas de formato, se reuqiere de ajustes.